# Equal-Weight S&P 500 Index Fund

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import yfinance as yf
from datetime import datetime, timedelta

In [39]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
sp500 = sp500['Symbol'].to_list()

In [52]:
stocks_combined = ' '.join(sp500)

stocks_data = yf.Tickers(stocks_combined)
stocks_data = stocks_data.tickers

In [66]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

for stock_ticker in stocks_data.values():

    response = stock_ticker.info

    if 'currentPrice' in response and 'marketCap' in response:
        new_row = pd.Series(
        [response['symbol'], response['currentPrice'], response['marketCap'], 'N/A'],
        index=my_columns
        )
    else:
        new_row = pd.Series(
        [response['symbol'], 0, 0, 'N/A'],
        index=my_columns
        )
    
    final_dataframe = pd.concat([final_dataframe, new_row.to_frame().T], ignore_index = True)


In [54]:
portfolio_size = input('Enter the amount of your portfolio')
val = 0.0

while(True):
    try:
        val = float(portfolio_size)
        break
    except ValueError:
        print('That is not a Number, Try Again!')
        portfolio_size = input('Enter the amount of your Portfolio')

In [55]:
position_size = val/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    if(float(final_dataframe.loc[i,'Stock Price']) != 0):
        final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / float(final_dataframe.loc[i,'Stock Price']))

In [56]:
writer = pd.ExcelWriter('Recommended Trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades',index=False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dolllar_format = writer.book.add_format(
    {
        'num_format': '$#,##0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

C:\Users\devag\AppData\Local\Temp\ipykernel_15796\1990354450.py:2: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  final_dataframe.to_excel(writer,'Recommended Trades',index=False)


In [57]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dolllar_format],
    'C': ['Market Capitalization', dolllar_format],
    'D': ['Number of Shares to Buy', integer_format] 
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',column_formats[column][0],string_format)
writer.close()